In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from statistics import mean 
import nltk

In [2]:
train = pd.read_csv("data/train_data.csv")
dev = pd.read_csv("data/dev_data.csv")
train["tweet"] = train["tweet"].astype(str)
dev["tweet"] = dev["tweet"].astype(str)

In [3]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


def lemmatize_sentence(tweet):
    word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()
    tokenized_sentence = word_tokenizer.tokenize(tweet)
    tokens = [lemmatize(token) for token in tokenized_sentence]
    return " ".join(tokens)
    
def remove_punct(tweet):
    punctuation = r"""[.\/#!$%\^&\*;\=\?\]\[:{}=\-_`~()\'\！\’"|]"""
    tweet = re.sub(punctuation, "", tweet)
    return tweet

def preprocess(tweet):
    tweet = remove_punct(tweet)
    #tweet = lemmatize_sentence(tweet)
    return tweet

In [4]:
train["tweet"].apply(preprocess)
dev["tweet"].apply(preprocess)

0                                        @handle Nevermind
1        @handle @handle I just think that developmenta...
2        The quasi dollar cab type shuttle dude is mad ...
3        Melissas cuban accent makes me laugh Its the c...
4        Studies show that adults who were bullied as c...
5        @handle betta believe itgood to see u pop up a...
6        Just started collaborating w developers at www...
7        Terra Industries Rises After CF Boosts Takeove...
8        Fabolous Not Planning Beanie Sigel Diss Record...
9                                        @handle good luck
10       @handle thanks for the hook up with Lori Shes ...
11       RT @handle Wow, we got 74 followers in our fir...
12       Damn RT @handle Crazy infographic, The Geograp...
13       Settling in to a nice pitchers duel, now Love ...
14                             I hate Saturday night drama
15       RT @handle Dear startups Too many of you are i...
16       @handle she was just here for mill valley film.

In [5]:
train.sort_values(by = "author_Id")
concatenated_train = train.groupby('author_Id').agg({'tweet': '\n'.join, 
                                                     'num_words':mean,
                                                     'ratio_stopwords':mean,
                                                     'ratio_upper':mean,
                                                     'average_word_length':mean,
                                                     'is_retweet': sum,
                                                     'is_modified_retweet': sum,
                                                     'ratio_emoticon':mean,
                                                     'ratio_punctuations':mean,
                                                     })
# ['tweet'].apply(' '.join).reset_index()


In [6]:
concatenated_train = concatenated_train.sample(frac=1).reset_index(drop=False)
concatenated_train

author_Id                                              tweet  num_words  \
0          3884  !In Tri-Cities Washington! When i get back hom...  14.000000   
1          4302  @handle Uh oh. What did you do? Why the formal...  14.500000   
2          2411  Fall is approaching! What are your favorite fa...  12.173913   
3           874  At mickey dees :)\nWork. :)\n@handle ggrr. Tmo...   9.529412   
4          9308  RT @handle: Feel the LOVE! Free Crashland Sket...  14.888889   
5          9703  @handle Cold? What kind of heels did you get?\...  16.222222   
6          4715  @handle When the people fear the government, t...  15.964286   
7          4234  Beautiful night in San Diego! :-) http\n@handl...  15.761905   
8           583  S. Jackson Ready for big yr: http\nDon't miss ...  17.680000   
9          8413  @handle lol I dunno...I'm going all out on thi...  13.000000   
10         8151  Working on http Work in progress.\nPainting my...  12.806452   
11         1166  @handle what are you asking ryan about? m&g .....  12.785714   
12         8321  Watching #BonesSeason5 premiere again with Meg...   9.133333   
13         2344  Fireworks and Music! See the St. Joseph Sympho...  18.642857   
14         4722  RT @handle got a "priority issues" survey from...  16.227273   
15         3592  Old-school portraits see resurgence online - h...  14.558140   
16         5803  Spicy tuna roll and cheese pastrami sandwich. ...  14.500000   
17         9567  RT @handle: @handle nice reader-review here! h...  17.272727   
18         9569  NOT SO IMPRESSED WITH NEW AUTOMATIC LITTER BOX...  15.266667   
19         6932  RankSurge.com: Oracle in World: Affordable Sea...  16.071429   
20         9664  FAIL.\nWhoo hoo!\nOk, seriously, this textbook...  11.466667   
21         6990  "Bring in the folks whose job it is to imagine...  15.206897   
22         9809  It's wet and dreary today, I might just go bac...  17.500000   
23         6902  @handle We're counted as late. There are some ...  15.400000   
24         4040  Volunteers - first meeting today from 5.30pm a...  17.727273   
25         4069  Reading the Greens: Four Questions From First-...   8.311475   
26         6651  I feel like Tom Cruise in Cocktail. At the Lif...   9.714286   
27         4366  RT @handle: Sir Roger Moore appears in campaig...  15.434783   
28            2  My cube buddy and I are having an Elton John m...  17.583333   
29         7942  Don't you hate days where you feel like an abj...  17.724138   
...         ...                                                ...        ...   
9264       3536  Fun Fact: Leonardo da Vinci could write with o...  15.900000   
9265       7742  Latest Sean Patrick Bowley HS football blog po...  11.851852   
9266       5017  aye this spam shit is just way too O.C.\n@hand...  10.233333   
9267       9867  Oh no, prayers for ECS and the families involv...  16.428571   
9268       6102  Doing it large tonight! We have great shows li...  21.439024   
9269       1513  Are You Ready For Retirement?http\nHOW TO CUST...  14.254545   
9270       5754  looks like tomorrow night we will be reviewing...  15.500000   
9271       6160  @handle Hey remember it was the tween awards a...  16.500000   
9272       5866  lets have a good day today people!\nfinna hit ...  10.625000   
9273       3880  Free tour of Ellis Creek Treatment plant and d...  15.687500   
9274       5610  Just realized the Jets and the Giants will be ...  19.236842   
9275       1997  Wall Street: Mixed week ends on a low note htt...  10.740741   
9276       1434  I want to read: Sacred Scars by Kathleen Duey ...  14.181818   
9277       4913  special #shoutout to @handle =) omg he's birff...  13.285714   
9278        748  RT @handle: @handle I like The F word & Kitche...  23.320000   
9279       6153  Bored to death is anything but. With film & HB...  23.827586   
9280       8957  @handle it just hit me....why weren't you the ...  13.666667   
9281       7551  @handle I optimize s

In [7]:
train_X = concatenated_train.loc[:, concatenated_train.columns != 'author_Id']
train_y = concatenated_train['author_Id']

In [8]:
vectorizer = TfidfVectorizer( 
                                decode_error=u'strict', strip_accents=None, 
                                preprocessor=None, 
                                tokenizer=None, analyzer=u'char', 
                                stop_words=None, 
                                ngram_range=(2, 3), max_df=1.0, 
                                min_df=1, max_features=8000, 
                                vocabulary=None, binary=False,  
                                norm=u'l2', use_idf=True, 
                                smooth_idf=True, sublinear_tf=False)

In [17]:
column_trans = ColumnTransformer([('text', vectorizer,'tweet')],
                                 remainder='drop')
     
    

In [18]:
dev_X = dev.loc[:, concatenated_train.columns != 'author_Id']

In [19]:
column_trans_train = column_trans.fit_transform(train_X, train_y)
#column_trans_dev = column_trans.fit_transform(dev_X)

In [20]:
column_trans_train.toarray().shape

(9294, 8000)

In [21]:
column_trans_dev_X = column_trans.transform(dev_X)

In [22]:
clf = LogisticRegression()
clf.fit(column_trans_train, train_y)
#clfprob = clf.predict_proba(dev_X)
predict_y = clf.predict(column_trans_dev_X)


/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [23]:
#print out he evaluate values
dev_y = dev['author_Id']
def classifier_report(true_y, predict_y):
    
    print("The accuracy: ",accuracy_score(true_y, predict_y))
    print("The precision: ", precision_score(true_y, predict_y, average = "macro"))
    print("The recall: ", recall_score(true_y, predict_y, average = "macro"))
    print("The f1-score: ", f1_score(true_y, predict_y, average = "macro"))
    
    return 
classifier_report(dev_y,predict_y)

The accuracy:  0.1171913951819829


/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The precision:  0.14300769376201364
The recall:  0.11202342942145678
The f1-score:  0.08797460753489846


/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda/envs/deepTF/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
import pickle

In [ ]:
filename = 'concatedFeature_logistic_model.sav'
pickle.dump(clf, open(filename, 'wb'))